<span type="title">内部类</span> | <span type="update">2018-07-23</span> - Version <span type="version">1.0</span>
    
    
<span type="intro"><p class="card-text">本章主要讲解内部类。</p></span>

# 内部类概要

## 内部类的定义、类型和调用

内部类是一种位于类内部的类。其允许将逻辑相关的代码放在一个“块”中，因此可以提高程序的清晰性。内部类不是“组合”，前者提供了一些类方法的实现，而“组合”则重在实例化现有的类，尽管从表面看来，它们在一个类中都持有其他的类。

内部类常常被用来**隐藏代码**，隐藏对于一个接口的具体实现过程。其可以和外围类进行通信，随意调用外围类的私有域和方法，即便声称了 `private`，但是外围类只有在实例化内部类之后才能访问内部类的动态域和方法。

内部类常常被用作一种创建**紧密但又清晰的类和类之间的关系**的一种方式。而在内部类之前，根据我们的介绍，类和类想要建立联系，必须通过 `相同的接口实现` 或者 `驴头不对马嘴的继承`。而通过内部类，我们可以将几个保存不同状态的类放置到一个类的内部，然后通过手动控制需要选用的类来改变这个外围类的状态。这常常在GUI或者控制器设计模式中较为常见。正因为此，我们才实现了真正的可以媲美C++的，更加优雅的基于类而不是接口的正经的多重继承。


```java
/Sequence.java
import java.util.Random; import static com.mazhangjing.Print.*;
/**
 * 选择器，包括三个方法 end() 是否结束；current()当前对象；next() 计数器加1，不返回值 */
interface Selector {
    boolean end();
    Object current();
    void next();
}
/**
 * 一个定常的序列类，数据保存在 sequence 数组中，next 表示当前序列长度
 * 私有类 sequenceselecter 实现了 selector，在内部操纵计数器 i，完成了判断是否结束、取回当前序列
 * 对应位置对象和计数器加1的操作。
 * */
public class Sequence {
    private Object[] sequence;
    private int next = 0;

    public Sequence(int size){
        sequence = new Object[size];
    }

    public void add(Object n){
        if (next < sequence.length){
            sequence[next++] = n; }
    }
    /**
     * 内部的 SenquenceSelecter 类可以自动获取外部类所有成员权限，不论是否私有。
     * 外部类需要一个方法来传递构造器构造好的内部类对象，在这里使用外部对象 getSelector()，或者
     * 使用 x.new SequenceSelecter() 创建，注意，不能写成 new Sequence.SequenceSelecter();
     * 外部类总是先于内部类进行对象的创建（其中的愿意那时内部类总要引用其所需要的
     * 外部类域或者方法）*/
    private class SequenceSelecter implements Selector {
        private int i = 0;
        public boolean end() {
            return i == sequence.length; }
        public Object current() {
            return sequence[i]; }
        public void next() {
            if (i < sequence.length) {i++;} }
        //对外部类的引用，返回值写外部类名，具体调用写类名.this，返回的是外部对象。
        public Sequence getFather(){
            return Sequence.this;
        }
    }
    //用以说明内部类的接口比外部类更加灵活的例子——可以对相同接口不同的实现
    private class reverseSelecter implements Selector {
        private int i = sequence.length;
        public boolean end() {
            return i == 0; }
        public Object current() {
            return sequence[i-1]; }
        public void next() {
            if (i > 0) {i--;} }
    }

    public Selector getSelector(){ return new SequenceSelecter();}

    public static void main(String[] args){
        Sequence x = new Sequence(10);
        CM2 a = new CM2("Hello World");
        CM2 b = new CM2("Hello CORKINE");
        for (int i = 0; i < 8; i++){ x.add(Integer.toString(i)); }
        x.add(a);
        x.add(b);
        //Selector s = x.getSelector(); 使用外部类方法创建内部类实例
        //需要注意，这里因为是一个接口，所以写成了Selector，如果不想向上转型，默认写作
        //Sequence.SequenceSelecter s = x.getSelector();
        //或者写成下面这样，使用.new直接创建内部类实例
        //因为外部类必须先存在，因此使用 x.new 而不是 new Sequence.SequenceSelecter()
        Sequence.SequenceSelecter s = x.new SequenceSelecter();
        Selector s2 = x.new reverseSelecter();
        while (!s2.end()){
            print(s2.current());
            s2.next();
        }
    }
}
class CM2 {
    private String s;
    CM2(String s){ this.s = s; }
    public String toString() { return this.s; }
}
```

在这个例子中，内部类 `SequenceSelector` 是对于接口 `Selector` 的一种实现。而和之前不同，没有使用外围类 `Sequence` 自身 `implement` 这个接口的原因是，其自身的实现很蹩脚，一个序列实现一个计数控制器从逻辑上来讲很别扭。但是在内部类之前，我们只有这一种选择。

**内部类的结构特征和初始化顺序**

注意这里的内部类语法，`private class XXX implement xxx` 和普通类没有什么区别，关键是其放置的位置，在外围类原本放置方法的地方。此外，当内部类进行构造的时候，其**需要外围类先进行构造**，然后才能引用外围类的域和方法。当构造内部类的时候，其其隐含了一个对于外围类的引用 `Sequence.this`，这用来指向外围类实例本身。再次强调，**内部类拥有外围类的一切访问权限，反过来则不成立。**

**内部类的类型、创建内部类实例的注意事项**

调用一个内部类需要注意内部类类型的问题。首先，如果这个内部类没有实现某个接口或者抽象类或者一般类，那么其类型应该是 `ClassFather.ClassInner` 比如这里的 `Sequence.SequenceSelector` 。而如果实现了某个接口，自然可以向上转型为 `Sequence` 类型。

其次，还要注意调用内部类语法问题。一般而言，内部类都会通过一个外围类的特定方法来返回其实例，比如这里的 `getSelector()` 注意这里的类型问题。但是，如果不通过这种方法，则需要通过 `ClassFatherInstance.new ClassInner()` 这种方式来调用。记得之前说过的，因为内部类可以随意调用外部类的域和方法，因此其必须等待外部类实例化之后才能实例化自身。它们之间的关系和继承中的初始化有些相似。因此，对于本例，使用 `x.new SequenctSelector()` 来创建内部类的实例。注意不能写成 `new Sequence.SequenceSelector()` 因为外部类需要初始化，并且内部类需要有一个外部类的对象的引用 `Sequence.this`。

## 内部类的必要性

**为什么外围类不自己去实现接口？ - 逻辑上不合适，内部代码容易混淆**

为什么外围类不自己实现这个 `Selector` 接口呢？可以，并且根据上一章的知识，它可以正确工作，但是，很多时候，让它自己实现一个接口太过于小题大做，并且实现接口的方法容易和其本身内部的方法造成混淆。

按照我们的例子，因为这里的Selector只用服务于 Sequence，它很小，并且最好向外界隐藏实现，作为一个助手即可。对于这种 `一一对应、任劳任怨` 的助手，应该有一种新的方式来处理 `类与类之间的这种奇妙关系`。因此有了内部类，在这种语法中，即有继承的那种域和方法的共享，并且清晰的理顺了逻辑（把和翻页相关的代码完全放在内部类中，提高了程序清晰性，并且因为内部类只能够通过实例访问，因此具有较高的独立性）。

**为什么内部类不继承外围类？或者不使用代理？ - 逻辑上不合适，难以复用代码**

对于传统的 `定义 - 实现`, 不论是通过对接口还是对于类的继承，有时候都不能够很好的映射现实中的问题。比如说在这里，如果不使用内部类，那么我就需要首先写一个 `SequenceSelector` 以对接口进行实现，然后因为 `Sequence` 需要使用这里的 `Selector`，并且是一个典型的 `is-a` 关系，所以就需要让 `Selector` 继承 `Sequence`（这样的话，在Selector内部就有了Sequence的方法，可以传出当前的Sequence内容），这样很别扭。那么还有一种方式是，使用代理，让 `Sequence` 被 `Selector` 所包裹，后者全权操纵前者，但是这样的结构难以重用（因为进行了实例化）。

因此，对于这种类和类之间的关系，使用 继承 并不合适。

**一言以蔽之：什么时候应该使用内部类？**

此外，另一个好处是，使用内部类，可以间接的实现像C++一样的多重继承，在Java中不允许直接多重继承自一个正常类，接口可以。使用内部类，可以间接的做到一个类对于很多类的多重继承。

一言以蔽之，**对于那些不平等的类和类之间的关系**，使用继承太驴头不对马嘴（虽然可以工作），使用接口通信逻辑上也难以讲通（虽然可以工作），因此，使用内部类，**一个大的，一堆小的**，逻辑清晰易懂，虽然手动调用麻烦了点，但是看起来和用起来其实差别不大。

# 在方法和作用域内的内部类

内部类不仅仅可以放置在外围类的内部，其也可以放置在外围类方法的内部，甚至是在某个作用域的内部，如下列代码：

```java
/FF.java
import java.util.Random; import static com.mazhangjing.Print.*;
public class FF {
    public I methodA(){
        //这个包裹在方法中的类的作用域仅限于此方法（除了这个方法外，在FF中不可
        // 访问classA），当方法调用完之后就被清理。但是因为有进一步的引用，所以
        //构造的对象现在还不会被垃圾回收。
        class classA implements I {
            public void i() {
                print("Implement of interface I in classA"); }
        }
        return new classA();
    }
    public void methodB() {
        Random rand = new Random();
        //被包裹在作用域内的类，出了作用域就不可用
        if (rand.nextInt(10)%2 == 0){
            class classB implements I {
                public void i() {
                    print("Implement of interface I in classB"); }
            }
            classB b = new classB();
            b.i();
        } else {print("You missed classB");} //在这里已经不可用
    }
    private class classC implements I {
        public void i() {
            print("Implement of interface I in classC"); }
    }
    public I methodC() {
        I x = this.new classC();
        return x;
    }
    public static void main(String[] args){
        FF x = new FF();
        x.methodA().i();
        x.methodB();
        x.methodC().i();
        ((FF.classC)(x.methodC())).i();
        //在其内部看不到向下转型的错误，当这句话在别的类中执行会出错，因为private的对象隐藏了接口的实现
        //这就是之前讲过的使用私有声明的内部类来隐藏接口实现
    }
}
interface I {
    void i();
}
```

可以看到，对于方法体内的内部类来说，当方法调用完，这个类就消失？真的吗？当然不是，如果存在指向这个类的引用，垃圾回收器不会管它，但如果像我们这样使用 `x.methodA().i()` 那么使用完后这个类就从内存中消失了。对于作用于内部的类而言，出了作用域也同样不可访问。内部类所在的位置决定了其可以访问的资源，比如，在 `methodA()` 中的这个内部类，其可以直接访问这个方法的参数，但是除了这个参数，外部如果声明一个 private 方法，它就不可访问了。

此外还需要注意，`classC` 在这里指的是一个正经的内部类。注意这里的 `private` 关键字，这意味着，如果在这个类外部访问这个内部类，可能吗？当然可能，使用其向上转型的类型 I 和方法 methodC，就可以直接访问。但是，不能使用 `FF.classC` 这种方法。这意味着什么呢？这意味着对于接口的实现，使用内部类的方法做到了完全的屏蔽，对于用户而言，其只知道交互的是一个 `I` 类型，而完全不知道是谁提供的实现，以及提供的实现是什么。